<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">
<h3 align="left"><font color='#f8251b'>Preprocess:</font></h3>

Balance the dataset

Apply log transformation to remove the skew

cap the outliers

Scale the dataset

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from feature_engine.outliers import Winsorizer
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.preprocessing import LabelEncoder


In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [24]:
df = pd.read_csv("../data/training.csv")

In [25]:
df.drop(columns=['EventId'], axis=1, inplace=True)

In [26]:
y = df['Label']
X = df.drop(columns=['Label'], axis=1)

# Split dataset

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y)

# Capping outliers

In [28]:
capper = Winsorizer(
    variables=['DER_mass_MMC', 'DER_mass_transverse_met_lep',
                'DER_mass_vis','DER_pt_h',
                'DER_mass_jet_jet','DER_deltar_tau_lep',
                'DER_pt_tot', 'DER_sum_pt',
                'DER_pt_ratio_lep_tau', 'PRI_tau_pt',
                'PRI_lep_pt', 'PRI_met',
                'PRI_met_sumet', 'PRI_jet_all_pt', 'Weight'],
    capping_method="quantiles",
    tail="both",
    fold=0.05,
)
capper.fit(X_train)



Winsorizer(capping_method='quantiles', fold=0.05, tail='both',
           variables=['DER_mass_MMC', 'DER_mass_transverse_met_lep',
                      'DER_mass_vis', 'DER_pt_h', 'DER_mass_jet_jet',
                      'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
                      'DER_pt_ratio_lep_tau', 'PRI_tau_pt', 'PRI_lep_pt',
                      'PRI_met', 'PRI_met_sumet', 'PRI_jet_all_pt', 'Weight'])

In [29]:
X_train.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt,Weight
count,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000,175000.000000
mean,-48.421401,49.159139,81.138032,57.982313,-708.445161,-601.388429,-709.376951,2.372637,18.859847,158.421638,1.436957,-0.128925,-709.008613,38.743552,-0.010575,-0.006002,46.664991,-0.019082,0.044502,41.707223,-0.009286,209.680844,0.978589,-348.508742,-399.446930,-399.452209,-692.420095,-709.143703,-709.139849,73.013097,1.646635
std,405.649703,35.272423,40.540864,63.950474,454.468504,657.593884,453.014209,0.781923,22.383271,115.508879,0.844507,1.193732,453.586271,22.518828,1.214420,1.817851,22.074180,1.265518,1.815162,33.163476,1.812518,126.365909,0.976933,533.035990,489.379049,489.374799,479.836223,453.376242,453.382004,97.655456,1.875320
min,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,46.104000,0.047000,-1.414000,-999.000000,20.000000,-2.499000,-3.142000,26.000000,-2.505000,-3.142000,0.109000,-3.142000,19.868000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-0.000000,0.001502
25%,78.192000,19.143750,59.378000,14.119000,-999.000000,-999.000000,-999.000000,1.810000,2.838000,77.519750,0.882000,-1.371000,-999.000000,24.608000,-0.925000,-1.574000,32.380000,-1.013000,-1.521000,21.356750,-1.572000,122.874000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000,0.018636
50%,104.973000,46.449500,73.744000,38.476000,-999.000000,-999.000000,-999.000000,2.491000,12.271500,120.655000,1.280000,-0.359000,-999.000000,31.815000,-0.021000,-0.032000,40.522500,-0.045000,0.087000,34.798000,-0.028000,179.619000,1.000000,38.934000,-1.873000,-2.089000,-999.000000,-999.000000,-999.000000,40.482500,1.153354
75%,130.573000,73.567000,92.254000,79.322750,0.487000,83.394500,-4.601750,2.961000,27.548250,200.601250,1.776250,1.224000,0.000000,45.066000,0.898000,1.573000,53.410000,0.958000,1.613000,51.827000,1.564000,263.378250,2.000000,75.329250,0.432000,0.501000,33.711250,-2.455000,-2.275000,109.964250,2.402390
max,1192.026000,690.075000,1349.351000,2834.999000,8.459000,4160.527000,16.690000,5.684000,2834.999000,1675.449000,19.773000,1.414000,1.000000,764.408000,2.497000,3.142000,560.271000,2.503000,3.142000,2842.617000,3.142000,1733.037000,3.000000,1120.573000,4.499000,3.141000,569.834000,4.500000,3.142000,1448.361000,7.817383


# Balance the dataset

In [30]:
smt = SMOTETomek()
X_smt, y_smt = smt.fit_resample(X_train, y_train)

In [31]:
X_smt.to_csv('../data/processed/X_train_balanced.csv')
X_test.to_csv('../data/processed/X_test.csv')

# Scale the dataset

In [32]:
scaler = RobustScaler()
scaler.fit(X_smt)

X_train_scaled = pd.DataFrame(scaler.transform(X_smt), columns=X_smt.columns) 
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns) 

In [33]:
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv')
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv')

# Encode target variable

In [34]:
print(y_smt.shape)
print(y_test.shape)

(224376,)
(75000,)


In [36]:
le = LabelEncoder()

y_train_encoded = pd.DataFrame(data= LabelEncoder().fit_transform(y_smt), columns=['Label'])
y_test_encoded = pd.DataFrame(data= LabelEncoder().fit_transform(y_test), columns=['Label'])

In [37]:
y_train_encoded.to_csv('../data/processed/y_train.csv')
y_test_encoded.to_csv('../data/processed/y_test.csv')